In [3]:
import pandas as pd 
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
import requests
from config_api import weather_apiKey as wkey

In [4]:

# Create list for weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + wkey

Beginning Data Retrieval     
-----------------------------


In [5]:
# Create 2000 random latitude and longitude coordinates 
lats = np.random.uniform(-90, 90, size = 2000)
lngs = np.random.uniform(-180, 180, size = 2000)
lats_lngs = zip(lats, lngs)
lats_lngs

# Add coordinates to a list 
latlngs_lst = list(lats_lngs)

In [6]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for latlng in latlngs_lst:
    city = citipy.nearest_city(latlng[0],latlng[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    # print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)

772

In [7]:

# Loop through city Lists 
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 
    if (i % 50 == 0 and i >= 50):
        set_count += 1 
        record_count = 1 
    
    # Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Run an API request for each of the cities
    try:
       
        # Parse the JSON data
        city_weather = requests.get(city_url).json()

        # Compile needed data
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        

        # Fill City Data list 
        city_data.append({"City": city.title(),
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         })
        
    #If an error occurs
    except: 
        print("--- City not found")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | semey
Processing Record 2 of Set 1 | ushuaia
Processing Record 3 of Set 1 | mataura
Processing Record 4 of Set 1 | sambava
Processing Record 5 of Set 1 | georgetown
Processing Record 6 of Set 1 | kapaa
Processing Record 7 of Set 1 | sentyabrskiy
--- City not found
Processing Record 8 of Set 1 | albany
Processing Record 9 of Set 1 | hobart
Processing Record 10 of Set 1 | lethem
Processing Record 11 of Set 1 | bluff
Processing Record 12 of Set 1 | chunoyar
Processing Record 13 of Set 1 | skalistyy
--- City not found
Processing Record 14 of Set 1 | isangel
Processing Record 15 of Set 1 | kamyshin
Processing Record 16 of Set 1 | hermanus
Processing Record 17 of Set 1 | iqaluit
Processing Record 18 of Set 1 | east london
Processing Record 19 of Set 1 | byron bay
Processing Record 20 of Set 1 | taolanaro
--- City not found
Processing Record 21 of Set 1 | provideniya
Processing Record 22 of Set 1 | saldanha
Processing Record 23 of Set 1 | rikitea
Processing Reco

In [8]:
len(city_data)

711

In [9]:

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Semey,KZ,2021-07-22 14:31:26,50.4111,80.2275,80.82,41,0,11.18
1,Ushuaia,AR,2021-07-22 14:30:16,-54.8000,-68.3000,40.66,75,75,9.22
2,Mataura,NZ,2021-07-22 14:31:27,-46.1927,168.8643,40.48,97,9,2.08
3,Sambava,MG,2021-07-22 14:31:27,-14.2667,50.1667,70.90,82,1,11.88
4,Georgetown,MY,2021-07-22 14:31:28,5.4112,100.3354,87.89,64,20,3.44
5,Kapaa,US,2021-07-22 14:31:29,22.0752,-159.3190,77.32,84,90,3.00
6,Albany,US,2021-07-22 14:31:29,42.6001,-73.9662,74.32,72,95,8.08
7,Hobart,AU,2021-07-22 14:31:29,-42.8794,147.3294,52.27,57,100,5.99
8,Lethem,GY,2021-07-22 14:31:29,3.3803,-59.7968,86.81,62,19,3.98
9,Bluff,NZ,2021-07-22 14:31:29,-46.6000,168.3333,40.80,81,69,12.66


In [19]:
# Create the output file 
output_data_file = '../weather_data/cities.csv'

# Export the City_Data into a CSV
city_data_df.to_csv(output_data_file, index_label='City_ID')